In [25]:
%pylab inline
import pandas as pd
import seaborn as sns
from collections import Counter, defaultdict
import pandas as pd
import glob
import os
import itertools
from matplotlib.legend import Legend
from matplotlib.lines import Line2D
from gcca import GCCA

import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import CCA
from ee546_utils import *
sns.set_style('white')
sns.set_context('paper', font_scale=2)

TISSUES_SHORT_FORM_MAP = {'lv': 'liver',
                          'skm': 'muscle',
                          'br': 'brain',
                          'ht': 'heart',
                          'kd': 'kidney',
                          'cb': 'cerebellum',
                          'lv': 'liver',
                          'ts': 'testes'
                         }

SPECIES_COMMON_NAME_MAP = {"bos_taurus": "Cow",
                           "rattus_norvegicus": "Rat",
                           "pongo_abelii": "Orangutan",
                           "monodelphis_domestica": "Opossum",
                           "macaca_mulatta": "Rhesus",
                           "pan_troglodytes": "Chimp",
                           "mus_musculus": "Mouse",
                           "homo_sapiens": "Human",
                           "gallus_gallus": "Chicken",
                           "ornithorhynchus_anatinus":  "Platypus",
                           "gorilla_gorilla": "Gorilla",
                           "pan_paniscus": "Bonobo"}

SPECIES_COMMON_NAME_MAP_SHORT = {"btaurus": "Cow",
                                 "rnorvegicus": "Rat",
                                 "pabelii": "Orangutan",
                                 "mdomestica": "Opposum",
                                 "mmulatta": "Rhesus",
                                 "ptroglodytes": "Chimp",
                                 "mmusculus": "Mouse",
                                 "hsapiens": "Human",
                                 "ggallus": "Chicken",
                                 "oanatinus":  "Platypus",
                                 "ggorilla": "Gorrila",
                                 "ppaniscus": "Bonobo"}
def species_short(name):
    return name[0] + name.split('_')[1]

Populating the interactive namespace from numpy and matplotlib


In [3]:
import scipy.linalg.interpolative as sli

In [4]:
A = np.random.normal(size=(200,10))


In [5]:
k = 3
idx, proj = sli.interp_decomp(A, k)

In [9]:
B = sli.reconstruct_skel_matrix(A, k, idx)

In [10]:
P = sli.reconstruct_interp_matrix(idx, proj)

In [12]:
B.shape

(200, 3)

In [13]:
P.shape

(3, 10)

In [14]:
P

array([[-0.23980771, -0.34633486, -0.19992984,  0.10473385, -0.11166316,
        -0.19330649,  0.08716   ,  1.        ,  0.        ,  0.        ],
       [-0.24446797,  0.34890309, -0.64153301, -0.07518289,  0.34135002,
         0.20184917,  0.22100707,  0.        ,  0.        ,  1.        ],
       [ 0.16500034,  0.13770571, -0.23160157, -0.11277886,  0.03112609,
         0.15257504,  0.25551029,  0.        ,  1.        ,  0.        ]])

In [16]:
ortho_pairwise = {}
for f in glob.glob('../ortholog_tsv/*.tsv'):
    if 'ortholog_one2one' in f:
        continue
    second_species = f.split('/')[-1].replace('.tsv', '').split('_')[-1]
    print(f)
    ortho_pairwise[f.split('/')[-1].replace('.tsv', '')] = pd.read_table(f)[['ensembl_gene_id', '{}_homolog_ensembl_gene'.format(second_species), '{}_homolog_orthology_type'.format(second_species)]].set_index('ensembl_gene_id')

../ortholog_tsv/mmusculus_btaurus.tsv
../ortholog_tsv/mmusculus_ggallus.tsv
../ortholog_tsv/mmusculus_ggorilla.tsv
../ortholog_tsv/mmusculus_hsapiens.tsv
../ortholog_tsv/mmusculus_mdomestica.tsv
../ortholog_tsv/mmusculus_mmulatta.tsv
../ortholog_tsv/mmusculus_oanatinus.tsv
../ortholog_tsv/mmusculus_pabelii.tsv
../ortholog_tsv/mmusculus_ppaniscus.tsv
../ortholog_tsv/mmusculus_ptroglodytes.tsv
../ortholog_tsv/mmusculus_rnorvegicus.tsv


In [18]:
kallisto_counts = {}
srp_species =  defaultdict(list)
for f in glob.glob('../cross-species-data-tpm/SRP016501-*_kallisto_gene_tables.tsv'):
    sample_name = f.split('/')[-1].replace('_kallisto_gene_tables.tsv', '')
    srp, species = sample_name.split('-')
    species = species[0] + species.split('_')[-1]
    kallisto_counts['{}_{}'.format(srp, species)] = pd.read_table(f)
    srp_species[srp].append(species)               

In [33]:
A = kallisto_counts['SRP016501_ggallus']
k = 3
idx, proj = sli.interp_decomp(A.values, k) 

In [35]:
B = sli.reconstruct_skel_matrix(A.values, k, idx)
P = sli.reconstruct_interp_matrix(idx, proj)

In [36]:
B.shape

(18389, 3)

In [37]:
A.shape

(18389, 27)

In [45]:
numpy.linalg.matrix_rank(A.values)

27

In [38]:
P.shape

(3, 27)

In [39]:
P

array([[ 0.04437096,  0.05776197,  0.25842608,  0.0806814 ,  0.00962803,
         0.0645376 , -0.11429262,  0.07554113,  0.12208608, -0.00552352,
        -0.01752855,  0.        ,  0.02542493, -0.04263414,  0.00757315,
        -0.19198097,  0.0261222 ,  0.16572877,  1.        , -0.00550958,
         0.08106287, -0.01128705,  0.        ,  0.07155796,  0.79131802,
         0.02125183,  0.08637423],
       [-0.09249109, -0.04414374,  0.11448751,  0.01797894,  0.40080295,
        -0.07268751,  0.12856069, -0.03611857, -0.02499104, -0.11900156,
        -0.08086286,  0.        ,  0.10552852,  0.39708728, -0.12934958,
         0.05184289, -0.1570135 , -0.02834744,  0.        , -0.03863533,
        -0.02022127, -0.00822038,  1.        , -0.03449927,  0.12657201,
        -0.05963721, -0.12552649],
       [ 0.30675194,  0.08353566,  0.42978387,  0.54307979,  0.12013083,
         0.07227845,  0.60132272,  0.02629173, -0.04157488,  0.23010551,
         0.22299028,  1.        ,  0.43941104,  0.2044

In [44]:
proj.shape

(3, 24)

In [23]:
species_wise_tissues = defaultdict(list)

for f in glob.glob('../cross-species-data-tpm//SRP016501-*_kallisto_gene_tables.tsv'):
    sample_name = f.split('/')[-1].replace('_kallisto_gene_tables.tsv', '')
    srp, species = sample_name.split('-')
    
    #species = species[0] + species.split('_')[-1]
    df = pd.read_table(f)
    tissues = sorted(set([x.split('_')[1] for x in df.columns]))
    for index, tissue in enumerate(tissues):
        if tissue in TISSUES_SHORT_FORM_MAP.keys():
            tissues[index] = TISSUES_SHORT_FORM_MAP[tissue]
            
    species_wise_tissues[species]+=list(tissues)
    
srp = 'SRP016501'
ortho_pairwise = {}
all_orthologs = pd.DataFrame()
for f in ['../ortholog_tsv/mmusculus_{}.tsv'.format(x) for x in map(lambda s: species_short(s), species_wise_tissues.keys())]:
    if 'mmusculus_mmusculus' in f:
        continue
    second_species = f.split('/')[-1].replace('.tsv', '').split('_')[-1]
    df = pd.read_table(f)[['ensembl_gene_id', '{}_homolog_ensembl_gene'.format(second_species), '{}_homolog_orthology_type'.format(second_species)]].set_index('ensembl_gene_id')
    df = df[df['{}_homolog_orthology_type'.format(second_species)]=='ortholog_one2one'].drop(columns=['{}_homolog_orthology_type'.format(second_species)])
    all_orthologs = all_orthologs.join(df, how='outer')

NameError: name 'species_short' is not defined